# Numerical Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *
#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover SPAM and Map

### Generate True Model, Full POVM and Inital

In [15]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = d
c1 = 0.9
c2 = 0.9

#prep error and full POVM error
spam_target = SPAM(d=d)

init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

spam_target = SPAM(d=d,
                  init = init_target,
                  povm = povm_target)

kraus_target = KrausMap(
                        U=generate_unitary(d=d), 
                        c=0.5, 
                        d=d, 
                        rank=rank-1,
                        spam = spam_target
                        )

### Generate Synthetic Data with Noise

In [16]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs_spam, _ = generate_pauliInput_circuits(n)

U_prep = inputs_spam
N_spam = U_prep.shape[0]

state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
state = apply_unitary(state, U_prep)
targets_spam = measurement(state, povm = spam_target.povm)

#add noise
targets_spam = add_noise_to_probs(targets_spam, 0.01)

inputs_map, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None, 
                                        N = 1800, 
                                        trace=False)
U_prep, U_basis = inputs_map

N_map = U_prep.shape[0]
state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
state = apply_unitary(state, U_prep)
state = kraus_target.apply_map(state)
targets_map = measurement(state, U_basis, spam_target.povm)

#add noise
targets_map = add_noise_to_probs(targets_map, 0.0316)

### Fit Model, Initial and Corruption Matrix

In [17]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(d=d,
                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 300,
                    verbose = False,
                    )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = N_spam,
                 verbose = False,
                )


kraus_model = KrausMap(
                       d = d, 
                       rank = d**2,
                       spam = spam_model,
                       )

  0%|          | 0/300 [00:00<?, ?it/s]

4.0923774930680634e-05


  0%|          | 0/1000 [00:00<?, ?it/s]

0.0010705312804176836


In [18]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

model = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = ProbabilityLoss(0.001),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )

model.train(inputs = inputs_map,
            targets = targets_map,
            #inputs_val = inputs_map,
            targets_val = [kraus_target],
            num_iter = 2000,
            N = 500,
            verbose = True,
            )

  0%|          | 0/2000 [00:00<?, ?it/s]

Step:0, train: 0.16017, val: 0.10299
Step:1, train: 0.16271, val: 0.10499
Step:2, train: 0.14972, val: 0.10709
Step:3, train: 0.15639, val: 0.10925
Step:4, train: 0.14925, val: 0.11147
Step:5, train: 0.15288, val: 0.11375
Step:6, train: 0.14931, val: 0.11610
Step:7, train: 0.14508, val: 0.11849
Step:8, train: 0.14382, val: 0.12092
Step:9, train: 0.14724, val: 0.12340
Step:10, train: 0.13990, val: 0.12595
Step:11, train: 0.14083, val: 0.12855
Step:12, train: 0.14034, val: 0.13122
Step:13, train: 0.13682, val: 0.13393
Step:14, train: 0.13458, val: 0.13674
Step:15, train: 0.12990, val: 0.13960
Step:16, train: 0.12774, val: 0.14251
Step:17, train: 0.12953, val: 0.14548
Step:18, train: 0.12585, val: 0.14850
Step:19, train: 0.12043, val: 0.15153
Step:20, train: 0.12516, val: 0.15461
Step:21, train: 0.11885, val: 0.15776
Step:22, train: 0.11618, val: 0.16098
Step:23, train: 0.11320, val: 0.16427
Step:24, train: 0.11199, val: 0.16760
Step:25, train: 0.10962, val: 0.17096
Step:26, train: 0.1095

Step:213, train: 0.00869, val: 0.69035
Step:214, train: 0.00934, val: 0.69152
Step:215, train: 0.00945, val: 0.69269
Step:216, train: 0.00893, val: 0.69387
Step:217, train: 0.00914, val: 0.69503
Step:218, train: 0.00870, val: 0.69619
Step:219, train: 0.00908, val: 0.69733
Step:220, train: 0.00880, val: 0.69847
Step:221, train: 0.00887, val: 0.69960
Step:222, train: 0.00894, val: 0.70071
Step:223, train: 0.00895, val: 0.70183
Step:224, train: 0.00919, val: 0.70294
Step:225, train: 0.00871, val: 0.70405
Step:226, train: 0.00870, val: 0.70515
Step:227, train: 0.00838, val: 0.70626
Step:228, train: 0.00881, val: 0.70736
Step:229, train: 0.00858, val: 0.70846
Step:230, train: 0.00834, val: 0.70955
Step:231, train: 0.00840, val: 0.71062
Step:232, train: 0.00869, val: 0.71168
Step:233, train: 0.00863, val: 0.71273
Step:234, train: 0.00849, val: 0.71377
Step:235, train: 0.00861, val: 0.71482
Step:236, train: 0.00856, val: 0.71586
Step:237, train: 0.00839, val: 0.71689
Step:238, train: 0.00823,

Step:425, train: 0.00637, val: 0.84016
Step:426, train: 0.00620, val: 0.84055
Step:427, train: 0.00604, val: 0.84094
Step:428, train: 0.00613, val: 0.84132
Step:429, train: 0.00625, val: 0.84169
Step:430, train: 0.00642, val: 0.84206
Step:431, train: 0.00620, val: 0.84242
Step:432, train: 0.00607, val: 0.84278
Step:433, train: 0.00613, val: 0.84315
Step:434, train: 0.00624, val: 0.84351
Step:435, train: 0.00613, val: 0.84387
Step:436, train: 0.00623, val: 0.84424
Step:437, train: 0.00616, val: 0.84460
Step:438, train: 0.00592, val: 0.84497
Step:439, train: 0.00608, val: 0.84533
Step:440, train: 0.00608, val: 0.84569
Step:441, train: 0.00629, val: 0.84603
Step:442, train: 0.00592, val: 0.84638
Step:443, train: 0.00625, val: 0.84673
Step:444, train: 0.00606, val: 0.84707
Step:445, train: 0.00602, val: 0.84741
Step:446, train: 0.00614, val: 0.84774
Step:447, train: 0.00599, val: 0.84806
Step:448, train: 0.00613, val: 0.84839
Step:449, train: 0.00623, val: 0.84872
Step:450, train: 0.00619,

Step:636, train: 0.00582, val: 0.89244
Step:637, train: 0.00568, val: 0.89259
Step:638, train: 0.00570, val: 0.89275
Step:639, train: 0.00587, val: 0.89292
Step:640, train: 0.00577, val: 0.89308
Step:641, train: 0.00563, val: 0.89324
Step:642, train: 0.00559, val: 0.89339
Step:643, train: 0.00584, val: 0.89354
Step:644, train: 0.00557, val: 0.89370
Step:645, train: 0.00563, val: 0.89385
Step:646, train: 0.00568, val: 0.89400
Step:647, train: 0.00584, val: 0.89416
Step:648, train: 0.00565, val: 0.89432
Step:649, train: 0.00587, val: 0.89447
Step:650, train: 0.00558, val: 0.89462
Step:651, train: 0.00570, val: 0.89477
Step:652, train: 0.00588, val: 0.89492
Step:653, train: 0.00589, val: 0.89507
Step:654, train: 0.00562, val: 0.89522
Step:655, train: 0.00582, val: 0.89537
Step:656, train: 0.00569, val: 0.89552
Step:657, train: 0.00571, val: 0.89568
Step:658, train: 0.00565, val: 0.89583
Step:659, train: 0.00565, val: 0.89599
Step:660, train: 0.00582, val: 0.89613
Step:661, train: 0.00575,

...

Step:1057, train: 0.00555, val: 0.92545
Step:1058, train: 0.00551, val: 0.92549
Step:1059, train: 0.00582, val: 0.92553
Step:1060, train: 0.00535, val: 0.92557
Step:1061, train: 0.00551, val: 0.92561
Step:1062, train: 0.00542, val: 0.92565
Step:1063, train: 0.00556, val: 0.92568
Step:1064, train: 0.00564, val: 0.92569
Step:1065, train: 0.00556, val: 0.92569
Step:1066, train: 0.00531, val: 0.92569
Step:1067, train: 0.00552, val: 0.92569
Step:1068, train: 0.00556, val: 0.92569
Step:1069, train: 0.00539, val: 0.92569
Step:1070, train: 0.00561, val: 0.92569
Step:1071, train: 0.00570, val: 0.92570
Step:1072, train: 0.00568, val: 0.92570
Step:1073, train: 0.00558, val: 0.92569
Step:1074, train: 0.00550, val: 0.92568
Step:1075, train: 0.00553, val: 0.92568
Step:1076, train: 0.00554, val: 0.92569
Step:1077, train: 0.00573, val: 0.92571
Step:1078, train: 0.00558, val: 0.92575
Step:1079, train: 0.00552, val: 0.92578
Step:1080, train: 0.00530, val: 0.92582
Step:1081, train: 0.00540, val: 0.92585


Step:1262, train: 0.00542, val: 0.93041
Step:1263, train: 0.00557, val: 0.93041
Step:1264, train: 0.00550, val: 0.93042
Step:1265, train: 0.00566, val: 0.93042
Step:1266, train: 0.00555, val: 0.93042
Step:1267, train: 0.00547, val: 0.93043
Step:1268, train: 0.00552, val: 0.93044
Step:1269, train: 0.00555, val: 0.93044
Step:1270, train: 0.00538, val: 0.93044
Step:1271, train: 0.00548, val: 0.93045
Step:1272, train: 0.00566, val: 0.93046
Step:1273, train: 0.00544, val: 0.93046
Step:1274, train: 0.00564, val: 0.93048
Step:1275, train: 0.00546, val: 0.93049
Step:1276, train: 0.00538, val: 0.93049
Step:1277, train: 0.00549, val: 0.93049
Step:1278, train: 0.00548, val: 0.93048
Step:1279, train: 0.00551, val: 0.93049
Step:1280, train: 0.00532, val: 0.93050
Step:1281, train: 0.00562, val: 0.93052
Step:1282, train: 0.00561, val: 0.93053
Step:1283, train: 0.00548, val: 0.93054
Step:1284, train: 0.00552, val: 0.93054
Step:1285, train: 0.00549, val: 0.93054
Step:1286, train: 0.00550, val: 0.93055


Step:1467, train: 0.00552, val: 0.93280
Step:1468, train: 0.00541, val: 0.93285
Step:1469, train: 0.00555, val: 0.93289
Step:1470, train: 0.00531, val: 0.93293
Step:1471, train: 0.00555, val: 0.93294
Step:1472, train: 0.00540, val: 0.93295
Step:1473, train: 0.00559, val: 0.93295
Step:1474, train: 0.00543, val: 0.93296
Step:1475, train: 0.00546, val: 0.93296
Step:1476, train: 0.00520, val: 0.93296
Step:1477, train: 0.00541, val: 0.93294
Step:1478, train: 0.00537, val: 0.93292
Step:1479, train: 0.00529, val: 0.93290
Step:1480, train: 0.00550, val: 0.93288
Step:1481, train: 0.00535, val: 0.93287
Step:1482, train: 0.00548, val: 0.93287
Step:1483, train: 0.00554, val: 0.93286
Step:1484, train: 0.00565, val: 0.93285
Step:1485, train: 0.00540, val: 0.93284
Step:1486, train: 0.00568, val: 0.93284
Step:1487, train: 0.00560, val: 0.93285
Step:1488, train: 0.00540, val: 0.93286
Step:1489, train: 0.00549, val: 0.93285
Step:1490, train: 0.00536, val: 0.93284
Step:1491, train: 0.00542, val: 0.93285


Step:1672, train: 0.00550, val: 0.93476
Step:1673, train: 0.00549, val: 0.93480
Step:1674, train: 0.00557, val: 0.93485
Step:1675, train: 0.00547, val: 0.93490
Step:1676, train: 0.00531, val: 0.93496
Step:1677, train: 0.00542, val: 0.93501
Step:1678, train: 0.00535, val: 0.93506
Step:1679, train: 0.00530, val: 0.93511
Step:1680, train: 0.00550, val: 0.93515
Step:1681, train: 0.00545, val: 0.93518
Step:1682, train: 0.00534, val: 0.93521
Step:1683, train: 0.00549, val: 0.93523
Step:1684, train: 0.00550, val: 0.93524
Step:1685, train: 0.00534, val: 0.93526
Step:1686, train: 0.00538, val: 0.93527
Step:1687, train: 0.00528, val: 0.93531
Step:1688, train: 0.00546, val: 0.93535
Step:1689, train: 0.00548, val: 0.93539
Step:1690, train: 0.00564, val: 0.93542
Step:1691, train: 0.00548, val: 0.93545
Step:1692, train: 0.00555, val: 0.93548
Step:1693, train: 0.00542, val: 0.93550
Step:1694, train: 0.00564, val: 0.93550
Step:1695, train: 0.00538, val: 0.93549
Step:1696, train: 0.00544, val: 0.93550


Step:1877, train: 0.00542, val: 0.93573
Step:1878, train: 0.00555, val: 0.93571
Step:1879, train: 0.00552, val: 0.93569
Step:1880, train: 0.00542, val: 0.93569
Step:1881, train: 0.00550, val: 0.93570
Step:1882, train: 0.00567, val: 0.93573
Step:1883, train: 0.00534, val: 0.93577
Step:1884, train: 0.00543, val: 0.93580
Step:1885, train: 0.00547, val: 0.93582
Step:1886, train: 0.00523, val: 0.93585
Step:1887, train: 0.00538, val: 0.93585
Step:1888, train: 0.00552, val: 0.93585
Step:1889, train: 0.00549, val: 0.93585
Step:1890, train: 0.00558, val: 0.93586
Step:1891, train: 0.00560, val: 0.93588
Step:1892, train: 0.00526, val: 0.93591
Step:1893, train: 0.00554, val: 0.93595
Step:1894, train: 0.00568, val: 0.93596
Step:1895, train: 0.00551, val: 0.93597
Step:1896, train: 0.00546, val: 0.93596
Step:1897, train: 0.00548, val: 0.93595
Step:1898, train: 0.00543, val: 0.93594
Step:1899, train: 0.00548, val: 0.93594
Step:1900, train: 0.00534, val: 0.93595
Step:1901, train: 0.00533, val: 0.93595


In [25]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

kraus_reg = KrausMap(
                       d = d, 
                       rank = d**2,
                       spam = spam_model,
                       )

model_reg = ModelQuantumMap(
                            q_map = kraus_reg,
                            loss = ProbabilityLoss(0.002),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

In [26]:
model_reg.train(inputs = inputs_map,
                targets = targets_map,
                #inputs_val = inputs_map,
                targets_val = [kraus_target],
                num_iter = 2000,
                N = 1800,
                verbose = True,
                )

  0%|          | 0/2000 [00:00<?, ?it/s]

Step:0, train: 0.22482, val: 0.09754
Step:1, train: 0.22256, val: 0.09964
Step:2, train: 0.22030, val: 0.10178
Step:3, train: 0.21804, val: 0.10397
Step:4, train: 0.21578, val: 0.10620
Step:5, train: 0.21353, val: 0.10847
Step:6, train: 0.21127, val: 0.11077
Step:7, train: 0.20902, val: 0.11313
Step:8, train: 0.20676, val: 0.11554
Step:9, train: 0.20451, val: 0.11799
Step:10, train: 0.20225, val: 0.12050
Step:11, train: 0.19998, val: 0.12307
Step:12, train: 0.19772, val: 0.12571
Step:13, train: 0.19544, val: 0.12841
Step:14, train: 0.19316, val: 0.13116
Step:15, train: 0.19087, val: 0.13397
Step:16, train: 0.18857, val: 0.13685
Step:17, train: 0.18626, val: 0.13979
Step:18, train: 0.18393, val: 0.14279
Step:19, train: 0.18160, val: 0.14584
Step:20, train: 0.17925, val: 0.14896
Step:21, train: 0.17690, val: 0.15214
Step:22, train: 0.17452, val: 0.15538
Step:23, train: 0.17213, val: 0.15869
Step:24, train: 0.16972, val: 0.16207
Step:25, train: 0.16729, val: 0.16552
Step:26, train: 0.1648

Step:213, train: 0.01755, val: 0.67462
Step:214, train: 0.01751, val: 0.67582
Step:215, train: 0.01747, val: 0.67702
Step:216, train: 0.01743, val: 0.67820
Step:217, train: 0.01738, val: 0.67939
Step:218, train: 0.01734, val: 0.68056
Step:219, train: 0.01730, val: 0.68173
Step:220, train: 0.01726, val: 0.68290
Step:221, train: 0.01722, val: 0.68406
Step:222, train: 0.01718, val: 0.68521
Step:223, train: 0.01715, val: 0.68636
Step:224, train: 0.01711, val: 0.68751
Step:225, train: 0.01707, val: 0.68864
Step:226, train: 0.01703, val: 0.68978
Step:227, train: 0.01699, val: 0.69090
Step:228, train: 0.01696, val: 0.69202
Step:229, train: 0.01692, val: 0.69314
Step:230, train: 0.01689, val: 0.69425
Step:231, train: 0.01685, val: 0.69535
Step:232, train: 0.01681, val: 0.69645
Step:233, train: 0.01678, val: 0.69755
Step:234, train: 0.01674, val: 0.69864
Step:235, train: 0.01671, val: 0.69972
Step:236, train: 0.01668, val: 0.70080
Step:237, train: 0.01664, val: 0.70188
Step:238, train: 0.01661,

Step:424, train: 0.01353, val: 0.83559
Step:425, train: 0.01352, val: 0.83604
Step:426, train: 0.01351, val: 0.83650
Step:427, train: 0.01350, val: 0.83695
Step:428, train: 0.01350, val: 0.83740
Step:429, train: 0.01349, val: 0.83785
Step:430, train: 0.01348, val: 0.83830
Step:431, train: 0.01347, val: 0.83874
Step:432, train: 0.01347, val: 0.83918
Step:433, train: 0.01346, val: 0.83962
Step:434, train: 0.01345, val: 0.84006
Step:435, train: 0.01345, val: 0.84050
Step:436, train: 0.01344, val: 0.84093
Step:437, train: 0.01343, val: 0.84137
Step:438, train: 0.01342, val: 0.84180
Step:439, train: 0.01342, val: 0.84223
Step:440, train: 0.01341, val: 0.84266
Step:441, train: 0.01340, val: 0.84308
Step:442, train: 0.01340, val: 0.84351
Step:443, train: 0.01339, val: 0.84393
Step:444, train: 0.01338, val: 0.84435
Step:445, train: 0.01338, val: 0.84477
Step:446, train: 0.01337, val: 0.84519
Step:447, train: 0.01336, val: 0.84560
Step:448, train: 0.01336, val: 0.84602
Step:449, train: 0.01335,

Step:635, train: 0.01264, val: 0.90046
Step:636, train: 0.01264, val: 0.90066
Step:637, train: 0.01264, val: 0.90086
Step:638, train: 0.01263, val: 0.90106
Step:639, train: 0.01263, val: 0.90126
Step:640, train: 0.01263, val: 0.90146
Step:641, train: 0.01263, val: 0.90165
Step:642, train: 0.01263, val: 0.90185
Step:643, train: 0.01262, val: 0.90205
Step:644, train: 0.01262, val: 0.90224
Step:645, train: 0.01262, val: 0.90243
Step:646, train: 0.01262, val: 0.90263
Step:647, train: 0.01262, val: 0.90282
Step:648, train: 0.01261, val: 0.90301
Step:649, train: 0.01261, val: 0.90320
Step:650, train: 0.01261, val: 0.90339
Step:651, train: 0.01261, val: 0.90358
Step:652, train: 0.01261, val: 0.90377
Step:653, train: 0.01260, val: 0.90396
Step:654, train: 0.01260, val: 0.90415
Step:655, train: 0.01260, val: 0.90434
Step:656, train: 0.01260, val: 0.90452
Step:657, train: 0.01260, val: 0.90471
Step:658, train: 0.01259, val: 0.90489
Step:659, train: 0.01259, val: 0.90508
Step:660, train: 0.01259,

...

Step:1055, train: 0.01224, val: 0.94611
Step:1056, train: 0.01224, val: 0.94616
Step:1057, train: 0.01224, val: 0.94621
Step:1058, train: 0.01224, val: 0.94627
Step:1059, train: 0.01224, val: 0.94632
Step:1060, train: 0.01224, val: 0.94637
Step:1061, train: 0.01224, val: 0.94642
Step:1062, train: 0.01224, val: 0.94648
Step:1063, train: 0.01224, val: 0.94653
Step:1064, train: 0.01224, val: 0.94658
Step:1065, train: 0.01223, val: 0.94663
Step:1066, train: 0.01223, val: 0.94668
Step:1067, train: 0.01223, val: 0.94674
Step:1068, train: 0.01223, val: 0.94679
Step:1069, train: 0.01223, val: 0.94684
Step:1070, train: 0.01223, val: 0.94689
Step:1071, train: 0.01223, val: 0.94694
Step:1072, train: 0.01223, val: 0.94699
Step:1073, train: 0.01223, val: 0.94704
Step:1074, train: 0.01223, val: 0.94709
Step:1075, train: 0.01223, val: 0.94714
Step:1076, train: 0.01223, val: 0.94719
Step:1077, train: 0.01223, val: 0.94724
Step:1078, train: 0.01223, val: 0.94729
Step:1079, train: 0.01223, val: 0.94734


Step:1260, train: 0.01219, val: 0.95440
Step:1261, train: 0.01219, val: 0.95443
Step:1262, train: 0.01219, val: 0.95446
Step:1263, train: 0.01219, val: 0.95449
Step:1264, train: 0.01219, val: 0.95452
Step:1265, train: 0.01219, val: 0.95455
Step:1266, train: 0.01219, val: 0.95458
Step:1267, train: 0.01219, val: 0.95461
Step:1268, train: 0.01219, val: 0.95464
Step:1269, train: 0.01218, val: 0.95467
Step:1270, train: 0.01218, val: 0.95470
Step:1271, train: 0.01218, val: 0.95472
Step:1272, train: 0.01218, val: 0.95475
Step:1273, train: 0.01218, val: 0.95478
Step:1274, train: 0.01218, val: 0.95481
Step:1275, train: 0.01218, val: 0.95484
Step:1276, train: 0.01218, val: 0.95487
Step:1277, train: 0.01218, val: 0.95490
Step:1278, train: 0.01218, val: 0.95493
Step:1279, train: 0.01218, val: 0.95496
Step:1280, train: 0.01218, val: 0.95498
Step:1281, train: 0.01218, val: 0.95501
Step:1282, train: 0.01218, val: 0.95504
Step:1283, train: 0.01218, val: 0.95507
Step:1284, train: 0.01218, val: 0.95510


Step:1465, train: 0.01216, val: 0.95915
Step:1466, train: 0.01216, val: 0.95917
Step:1467, train: 0.01216, val: 0.95919
Step:1468, train: 0.01216, val: 0.95920
Step:1469, train: 0.01216, val: 0.95922
Step:1470, train: 0.01216, val: 0.95924
Step:1471, train: 0.01216, val: 0.95926
Step:1472, train: 0.01216, val: 0.95927
Step:1473, train: 0.01216, val: 0.95929
Step:1474, train: 0.01216, val: 0.95931
Step:1475, train: 0.01216, val: 0.95933
Step:1476, train: 0.01216, val: 0.95934
Step:1477, train: 0.01216, val: 0.95936
Step:1478, train: 0.01216, val: 0.95938
Step:1479, train: 0.01216, val: 0.95939
Step:1480, train: 0.01216, val: 0.95941
Step:1481, train: 0.01216, val: 0.95943
Step:1482, train: 0.01216, val: 0.95944
Step:1483, train: 0.01216, val: 0.95946
Step:1484, train: 0.01216, val: 0.95948
Step:1485, train: 0.01216, val: 0.95949
Step:1486, train: 0.01216, val: 0.95951
Step:1487, train: 0.01216, val: 0.95953
Step:1488, train: 0.01216, val: 0.95954
Step:1489, train: 0.01216, val: 0.95956


Step:1670, train: 0.01215, val: 0.96197
Step:1671, train: 0.01215, val: 0.96198
Step:1672, train: 0.01215, val: 0.96199
Step:1673, train: 0.01215, val: 0.96200
Step:1674, train: 0.01215, val: 0.96202
Step:1675, train: 0.01215, val: 0.96203
Step:1676, train: 0.01215, val: 0.96204
Step:1677, train: 0.01215, val: 0.96205
Step:1678, train: 0.01215, val: 0.96206
Step:1679, train: 0.01215, val: 0.96207
Step:1680, train: 0.01215, val: 0.96208
Step:1681, train: 0.01215, val: 0.96209
Step:1682, train: 0.01215, val: 0.96210
Step:1683, train: 0.01215, val: 0.96211
Step:1684, train: 0.01215, val: 0.96212
Step:1685, train: 0.01215, val: 0.96213
Step:1686, train: 0.01215, val: 0.96214
Step:1687, train: 0.01215, val: 0.96215
Step:1688, train: 0.01215, val: 0.96216
Step:1689, train: 0.01215, val: 0.96217
Step:1690, train: 0.01215, val: 0.96218
Step:1691, train: 0.01215, val: 0.96219
Step:1692, train: 0.01215, val: 0.96220
Step:1693, train: 0.01215, val: 0.96221
Step:1694, train: 0.01215, val: 0.96222


Step:1875, train: 0.01214, val: 0.96371
Step:1876, train: 0.01214, val: 0.96371
Step:1877, train: 0.01214, val: 0.96372
Step:1878, train: 0.01214, val: 0.96373
Step:1879, train: 0.01214, val: 0.96373
Step:1880, train: 0.01214, val: 0.96374
Step:1881, train: 0.01214, val: 0.96375
Step:1882, train: 0.01214, val: 0.96375
Step:1883, train: 0.01214, val: 0.96376
Step:1884, train: 0.01214, val: 0.96376
Step:1885, train: 0.01214, val: 0.96377
Step:1886, train: 0.01214, val: 0.96378
Step:1887, train: 0.01214, val: 0.96378
Step:1888, train: 0.01214, val: 0.96379
Step:1889, train: 0.01214, val: 0.96380
Step:1890, train: 0.01214, val: 0.96380
Step:1891, train: 0.01214, val: 0.96381
Step:1892, train: 0.01214, val: 0.96382
Step:1893, train: 0.01214, val: 0.96382
Step:1894, train: 0.01214, val: 0.96383
Step:1895, train: 0.01214, val: 0.96384
Step:1896, train: 0.01214, val: 0.96384
Step:1897, train: 0.01214, val: 0.96385
Step:1898, train: 0.01214, val: 0.96385
Step:1899, train: 0.01214, val: 0.96386
